In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [3]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [4]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [5]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1153.0


In [6]:
# Confirmados PCR Huelva capital
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  306.0


In [7]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PCR 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [8]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 2993 personas en los últimos 7 días 

Un positivo PCR cada 1731 personas en los últimos 14 días


In [9]:
filtro = ((df['Medida'] == 'Confirmados PCR')| (df['Medida'] == 'Confirmados PCR 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PCR 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [10]:
# Reorder columns
tabla = tabla[['Confirmados PCR', 'Confirmados PCR 7 días', 'Confirmados PCR 14 días', 'Población']]

In [11]:
tabla['tasa7'] = tabla['Confirmados PCR 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PCR 14 días']/tabla['Población']*100000

In [12]:
tabla['Diferencia PCR'] = tabla['Confirmados PCR'] - dfAnterior['Confirmados PCR']

In [13]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PCR'] > 0].sort_values('Diferencia PCR', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Huelva,1153.0,159.0,336.0,510743.0,31.131117,65.786511,12.0
Huelva-Costa,617.0,92.0,190.0,288115.0,31.931694,65.945890,5.0
Aroche,8.0,4.0,6.0,3073.0,130.165962,195.248942,3.0
Condado-Campiña,349.0,48.0,109.0,155057.0,30.956358,70.296730,3.0
Sierra de Huelva-Andévalo Central,167.0,17.0,28.0,67571.0,25.158722,41.437895,3.0
Palos de la Frontera,26.0,5.0,18.0,11289.0,44.290903,159.447250,2.0
Ayamonte,38.0,12.0,23.0,20946.0,57.290175,109.806168,1.0
Cartaya,21.0,2.0,7.0,19974.0,10.013017,35.045559,1.0
Huelva (capital),306.0,48.0,83.0,143663.0,33.411526,57.774096,1.0


In [14]:
if tabla[tabla['Diferencia PCR'] > 0]['Diferencia PCR'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [15]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PCR,Confirmados PCR 7 días,Confirmados PCR 14 días,Población,tasa7,tasa14,Diferencia PCR
Lugar de residencia,,,,,,,
Cabezas Rubias,9.0,2.0,7.0,730.0,273.972603,958.904110,0.0
Villalba del Alcor,42.0,4.0,15.0,3338.0,119.832235,449.370881,0.0
Bollullos Par del Condado,46.0,12.0,28.0,14272.0,84.080717,196.188341,0.0
Zalamea la Real,20.0,1.0,6.0,3068.0,32.594524,195.567145,0.0
Aroche,8.0,4.0,6.0,3073.0,130.165962,195.248942,3.0
Palos de la Frontera,26.0,5.0,18.0,11289.0,44.290903,159.447250,2.0
Isla Cristina,58.0,6.0,25.0,21264.0,28.216704,117.569601,0.0
Nerva,20.0,6.0,6.0,5235.0,114.613181,114.613181,0.0
Ayamonte,38.0,12.0,23.0,20946.0,57.290175,109.806168,1.0
